# 第九章 数据库读写数据

In [11]:
import pandas as pd
import sqlite3

## 9.1 读取sqlite 数据库

到目前为止，我们只涉及从 CSV 文件中读取数据。 这是一个存储数据的常见方式，但有很多其它方式！ Pandas 可以从 `HTML，JSON，SQL，Excel（!!!），HDF5，Stata` 和其他一些东西中读取数据。 在本章中，我们将讨论从 SQL 数据库读取数据。

您可以使用`pd.read_sql`函数从 SQL 数据库读取数据。 `read_sql`将自动将 SQL 列名转换为DataFrame列名。

read_sql需要 2 个参数：SELECT语句和数据库连接对象。 这是极好的，因为它意味着你可以从任何种类的 SQL 数据库读取 - 无论是 `MySQL，SQLite，PostgreSQL `或其他东西。

此示例从 SQLite 数据库读取，但任何其他数据库将以相同的方式工作。

In [12]:
con = sqlite3.connect("../data/weather_2012.sqlite")
df = pd.read_sql("SELECT * from weather_2012 LIMIT 3", con)
df


,id,date_time,temp
0,1,2012-01-01 00:00:00,-1.8
1,2,2012-01-01 01:00:00,-1.8
2,3,2012-01-01 02:00:00,-1.8


`read_sql`不会自动将主键`（id）`设置为`DataFrame`的索引。 你可以通过向`read_sql`添加一个`index_col`参数来实现。

如果你大量使用read_csv，你可能已经看到它有一个index_col参数。 这个行为是一样的。

In [13]:
df = pd.read_sql("select * from weather_2012 limit 3", con, index_col='id')
df 

,date_time,temp
id,,
1,2012-01-01 00:00:00,-1.8
2,2012-01-01 01:00:00,-1.8
3,2012-01-01 02:00:00,-1.8


如果希望`DataFrame`由多个列索引，可以将列的列表提供给`index_col`：

In [14]:
df = pd.read_sql("select * from weather_2012 limit 3",con, index_col=['id','date_time'])
df 

,,temp
id,date_time,
1,2012-01-01 00:00:00,-1.8
2,2012-01-01 01:00:00,-1.8
3,2012-01-01 02:00:00,-1.8


## 9.2 写入 SQLite 数据库
Pandas 拥有`write_frame`函数，它从`DataFrame`创建一个数据库表。 现在这只适用于 `SQLite` 数据库。 让我们使用它，来将我们的 2012 天气数据转换为 SQL。

你会注意到这个函数在`pd.io.sql`中。 在`pd.io`中有很多有用的函数，用于读取和写入各种类型的数据，值得花一些时间来探索它们。 （请参阅文档！）

In [15]:
weather_df = pd.read_csv('../data/weather_2012.csv')
con = sqlite3.connect('../data/test_db.sqlite')
con.execute("drop table if exists weather_2012")
weather_df.to_sql("weather_2012",con)

8784

我们现在可以从test_db.sqlite中的weather_2012表中读取数据，我们看到我们得到了相同的数据：

In [16]:
con = sqlite3.connect('../data/test_db.sqlite')
df = pd.read_sql("select * from weather_2012 limit 3",con)
df 

,index,Date/Time,Temp (C),Dew Point Temp (C),Rel Hum (%),Wind Spd (km/h),Visibility (km),Stn Press (kPa),Weather
0,0,2012-01-01 00:00:00,-1.8,-3.9,86,4,8.0,101.24,Fog
1,1,2012-01-01 01:00:00,-1.8,-3.7,87,4,8.0,101.24,Fog
2,2,2012-01-01 02:00:00,-1.8,-3.4,89,7,4.0,101.26,"Freezing Drizzle,Fog"


在数据库中保存数据的好处在于，可以执行任意的 SQL 查询。 这非常酷，特别是如果你更熟悉 SQL 的情况下。 以下是Weather列排序的示例：

In [17]:
con = sqlite3.connect("../data/test_db.sqlite")
df = pd.read_sql("SELECT * from weather_2012 ORDER BY Weather LIMIT 3", con)
df

,index,Date/Time,Temp (C),Dew Point Temp (C),Rel Hum (%),Wind Spd (km/h),Visibility (km),Stn Press (kPa),Weather
0,67,2012-01-03 19:00:00,-16.9,-24.8,50,24,25.0,101.74,Clear
1,114,2012-01-05 18:00:00,-7.1,-14.4,56,11,25.0,100.71,Clear
2,115,2012-01-05 19:00:00,-9.2,-15.4,61,7,25.0,100.80,Clear


如果你有一个 PostgreSQL 数据库或 MySQL 数据库，从它读取的工作方式与从 SQLite 数据库读取完全相同。 使用psycopg2.connect()或MySQLdb.connect()创建连接，然后使用

In [ ]:
pd.read_sql("SELECT whatever from your_table", con)

## 9.3 连接到其它类型的数据库
为了连接到 MySQL 数据库：

注：为了使其正常工作，你需要拥有 MySQL/PostgreSQL 数据库，并带有正确的localhost，数据库名称，以及其他。

In [ ]:
import MySQLdb 
con = MySQLdb.connect(host="localhost", db="test")

为了连接到 PostgreSQL 数据库：

In [ ]:
import psycopg2 
con = psycopg2.connect(host="localhost")